Here I just want to test out my various algorithms to make sure they're actually working properly. 

In [1]:
from funcs import *
from multi_funcs import *

In [88]:
df = mdf_np(10,5)

In [89]:
df

,0,1,2,3,4,student_id,applications,k,matched,rank1,rank2,rank3,rank4,rank5,underdemanded
0,9,4,5,6,1,0,0,0,False,0.168414,0.600476,0.867827,0.399127,0.383325,True
1,6,2,4,9,7,1,0,0,False,0.804337,0.611048,0.389559,0.672483,0.376118,True
2,2,3,5,4,1,2,0,0,False,0.388096,0.436332,0.626571,0.306159,0.407220,True
3,9,2,1,8,3,3,0,0,False,0.731319,0.956535,0.729539,0.920242,0.368714,True
4,2,5,7,0,4,4,0,0,False,0.646058,0.055819,0.460726,0.654360,0.615204,True
5,5,8,2,3,6,5,0,0,False,0.814080,0.611120,0.634506,0.871937,0.869827,True
6,7,0,5,8,6,6,0,0,False,0.953368,0.622858,0.914129,0.983214,0.815712,True
7,4,7,6,1,3,7,0,0,False,0.703440,0.944318,0.963672,0.383070,0.393466,True
8,4,8,2,6,3,8,0,0,False,0.678899,0.042957,0.776123,0.364161,0.026413,True
9,3,5,8,7,2,9,0,0,False,0.389475,0.974985,0.855871,0.979928,0.879541,True


In [90]:
df[0] = np.arange(10)

In [91]:
df = df.iloc[:6,:]

In [92]:
df[0] = [0,1,2,3,4,4]
df['rank1'] = [0,0,0,0,0,1]
df['rank2'] = [1.1, 1.1, 1.1, 1.1, 1.1, 0]
df[1] = [1,2,3,4,0,5]

In [93]:
df['k'] = 1
df

,0,1,2,3,4,student_id,applications,k,matched,rank1,rank2,rank3,rank4,rank5,underdemanded
0,0,1,5,6,1,0,0,1,False,0,1.1,0.867827,0.399127,0.383325,True
1,1,2,4,9,7,1,0,1,False,0,1.1,0.389559,0.672483,0.376118,True
2,2,3,5,4,1,2,0,1,False,0,1.1,0.626571,0.306159,0.407220,True
3,3,4,1,8,3,3,0,1,False,0,1.1,0.729539,0.920242,0.368714,True
4,4,0,7,0,4,4,0,1,False,0,1.1,0.460726,0.654360,0.615204,True
5,4,5,2,3,6,5,0,1,False,1,0.0,0.634506,0.871937,0.869827,True


In [87]:
def run_gale_shapley(sp, k = 3):
    '''Runs the Gale-Shapley Algorithm for a particular dataframe.
    ______
    sp: dataframe of student preferences and school scores over those students.    
    '''
    r_apps = 0
    i=0
    mask = [1]
    while r_apps < k and sum(mask) > 0: ## note that there are two ways to end the cycle. 1 all unmatched students have applied to 3 schools. 2. There are no unmatched students.
        ## Step 1: Find k
        sp = find_k(sp)
        #sp['underdemanded'] = np.where(sp['k']>1, False, sp['underdemanded'])

        ## Step 2: Find matches
        matches = find_matches_3(sp) ## finds first round of matches.

        ## Step 3: Create mask and apply to matched column
        mask = create_mask(matches, sp, k) ## Creates a mask of students who have not been assigned to a school.
        sp.loc[:,'matched'] = mask

        ## Step 4: Update the underdemanded column
        condition = np.logical_or(sp['k'] > 1, sp['underdemanded'] == False)
        sp['underdemanded'] = np.where(np.logical_and(condition,sp['matched'] == False), False, sp['underdemanded'])
        sp['underdemanded'] = np.where(sp['matched'] == True, True, sp['underdemanded'])


        ## Step 5: Shift Values Over for unmatched
        sp = shift(sp, mask, k)

        ## Step 6: Update N, to reflect total number of matches for each school. 
        #sp['N'] += matches.set_index(0)['k'].reindex(sp[0], fill_value=0).reset_index(drop=True)

        ## Step 7: Remove all rejects who have applied to k schools:
        sp = sp[sp['applications'] < k] # If you've been rejected from three schools, you're out. We also don't want you to stay in the dataset. 
        sp.reset_index(drop=True, inplace=True) # Reset the index
        i+=1
    return sp, i

In [94]:
g = run_gale_shapley(df)

In [95]:
g

(   0  1  2  3  4  student_id  applications  k  matched  rank1     rank2  \
 0  1  5  5  6  1           0             1  1    False    1.1  0.867827   
 1  2  4  4  9  7           1             1  1    False    1.1  0.389559   
 2  3  5  5  4  1           2             1  1    False    1.1  0.626571   
 3  4  1  1  8  3           3             1  1    False    1.1  0.729539   
 4  0  7  7  0  4           4             1  1    False    1.1  0.460726   
 5  5  2  2  3  6           5             1  1    False    0.0  0.634506   
 
       rank3     rank4     rank5  underdemanded  
 0  0.867827  0.399127  0.383325          False  
 1  0.389559  0.672483  0.376118          False  
 2  0.626571  0.306159  0.407220          False  
 3  0.729539  0.920242  0.368714          False  
 4  0.460726  0.654360  0.615204          False  
 5  0.634506  0.871937  0.869827           True  ,
 7)